In [1]:
import boto3
import json
import numpy as np
import xmltodict
import pandas as pd

In [2]:
def get_config(filepath='secrets.json'):
    config = {}
    with open(filepath, 'r') as f:
        config = json.load(f)
    return config

In [3]:
region_name = 'us-east-1'
config = get_config()
aws_access_key_id = config['aws_access_key_id']
aws_secret_access_key = config['aws_secret_access_key']

endpoint_url = 'https://mturk-requester-sandbox.us-east-1.amazonaws.com'

# Uncomment this line to use in production
# endpoint_url = 'https://mturk-requester.us-east-1.amazonaws.com'

client = boto3.client(
    'mturk',
    endpoint_url=endpoint_url,
    region_name=region_name,
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
)

# This will return $10,000.00 in the MTurk Developer Sandbox
print(client.get_account_balance()['AvailableBalance'])

10000.00


# Approve Assignments

In [9]:
#Get HIT Id from the create HIT notebook
hitId = '3NC6WP7WKPSYEY4BOLBJQBXSHT0WWK'

assignmentsDF = pd.read_csv('data/{}.csv'.format(hitId))
for index, assignment in assignmentsDF.iterrows():
    print(assignment['AssignmentId'])
    client.approve_assignment(AssignmentId=assignment['AssignmentId'], RequesterFeedback="Thank you.")

3IFS6Q0HJRRPT61R5125GFHEQY3SI6
3OWEPKL08IKVO9SQEEHMQWHZPJKN7U


# Send Bonus

In [6]:
bonusAmount = '0.05'
reasons='Thank you for your feedback (via the Post Survey in the HIT). It will help improve our future tasks.'

# "WorkerId": String,
# "AssignmentId": String,
# "BonusAmount": String,
# "Reason": String,
# "UniqueRequestToken": String

for index, assignment in assignmentsDF.iterrows():
    print(assignment['WorkerId'], assignment['AssignmentId'])
    client.send_bonus(WorkerId=assignment['WorkerId'], 
                      AssignmentId=assignment['AssignmentId'], 
                      BonusAmount=bonusAmount, 
                      Reason=reasons, 
                      UniqueRequestToken='SendingBonus'+assignment['AssignmentId'])

{'ResponseMetadata': {'RequestId': 'ed80088c-5b54-4b41-a8a1-6c243dd91ef7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ed80088c-5b54-4b41-a8a1-6c243dd91ef7',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'date': 'Wed, 16 Mar 2022 02:16:34 GMT'},
  'RetryAttempts': 0}}

# Reject Assignments

In [ ]:
# assignmentsDF = pd.read_csv('data/{}.csv'.format(hitId))
# for index, assignment in assignmentsDF.iterrows():
#     print(assignment['AssignmentId'])
#     client.reject_assignment(AssignmentId=assignment['AssignmentId'], RequesterFeedback="Your responses consisted of all zeros.")